In [13]:
# Generate a mesh using dolfinx

from mpi4py import MPI
from dolfinx import mesh
points = [(0.0, 0.0), (3.0, 1.0)]
# Define the number of cells in the x and y directions
n = (30,10)
# Create the mesh
domain = mesh.create_rectangle(MPI.COMM_WORLD, points, n, cell_type=mesh.CellType.triangle)

# define finite element function space
from dolfinx.fem import functionspace
V = functionspace(domain, ("Lagrange", 1))

# identify the boundary (create facet to cell connectivity required to determine boundary facets)
tdim = domain.topology.dim
fdim = tdim - 1
domain.topology.create_connectivity(fdim, tdim)
boundary_facets = mesh.exterior_facet_indices(domain.topology)

# create a mesh function to mark the boundary facets
import numpy as np
from dolfinx import default_scalar_type
from dolfinx.fem import (Function, dirichletbc, locate_dofs_geometrical)
from dolfinx.fem.petsc import LinearProblem

def u_exact(x):
    return x[1]

def boundary_D(x):
    return np.logical_or(np.isclose(x[1], 0), np.isclose(x[1], 1))

dofs_D = locate_dofs_geometrical(V, boundary_D)

# boundary conditions
u_bc = Function(V)
u_bc.interpolate(u_exact)
bc = dirichletbc(u_bc, dofs_D)

# trial and test functions
import ufl
u = ufl.TrialFunction(V)
v = ufl.TestFunction(V)

# source term
from dolfinx import default_scalar_type
from dolfinx import fem
f = fem.Constant(domain, default_scalar_type(0))

# variational problem
a = ufl.dot(ufl.grad(u), ufl.grad(v)) * ufl.dx
L = f * v * ufl.dx

# assemble the system
from dolfinx.fem.petsc import LinearProblem
problem = LinearProblem(a, L, bcs=[bc], petsc_options={"ksp_type": "preonly", "pc_type": "lu"})
uh = problem.solve()

In [14]:
# plot the solution
import pyvista
pyvista.OFF_SCREEN = False
print(pyvista.global_theme.jupyter_backend)

from dolfinx import plot
pyvista.start_xvfb()
domain.topology.create_connectivity(tdim, tdim)
topology, cell_types, geometry = plot.vtk_mesh(domain, tdim)
grid = pyvista.UnstructuredGrid(topology, cell_types, geometry)

plotter = pyvista.Plotter()
plotter.add_mesh(grid, show_edges=True)
plotter.view_xy()
if not pyvista.OFF_SCREEN:
    plotter.show()
else:
    figure = plotter.screenshot("fundamentals_mesh.png")


trame


Widget(value='<iframe src="http://localhost:33717/index.html?ui=P_0x7fbb28bb5c30_8&reconnect=auto" class="pyvi…

In [15]:
u_topology, u_cell_types, u_geometry = plot.vtk_mesh(V)

u_grid = pyvista.UnstructuredGrid(u_topology, u_cell_types, u_geometry)
u_grid.point_data["u"] = uh.x.array.real
u_grid.set_active_scalars("u")
u_plotter = pyvista.Plotter()
u_plotter.add_mesh(u_grid, show_edges=True)
u_plotter.view_xy()
if not pyvista.OFF_SCREEN:
    u_plotter.show()

Widget(value='<iframe src="http://localhost:33717/index.html?ui=P_0x7fbaa5eb9e70_9&reconnect=auto" class="pyvi…

In [16]:
warped = u_grid.warp_by_scalar()
plotter2 = pyvista.Plotter()
plotter2.add_mesh(warped, show_edges=True, show_scalar_bar=True)
if not pyvista.OFF_SCREEN:
    plotter2.show()

Widget(value='<iframe src="http://localhost:33717/index.html?ui=P_0x7fbb537e31c0_10&reconnect=auto" class="pyv…

In [17]:
warped = u_grid.warp_by_scalar()
plotter2 = pyvista.Plotter()
plotter2.add_mesh(warped, show_edges=True, show_scalar_bar=True)
if not pyvista.OFF_SCREEN:
    plotter2.show()


Widget(value='<iframe src="http://localhost:33717/index.html?ui=P_0x7fba4de6b6a0_11&reconnect=auto" class="pyv…

In [18]:
from dolfinx import io
from pathlib import Path
results_folder = Path("results")
results_folder.mkdir(exist_ok=True, parents=True)
filename = results_folder / "fundamentals"
with io.VTXWriter(domain.comm, filename.with_suffix(".bp"), [uh]) as vtx:
    vtx.write(0.0)
with io.XDMFFile(domain.comm, filename.with_suffix(".xdmf"), "w") as xdmf:
    xdmf.write_mesh(domain)
    xdmf.write_function(uh)